zsh:1: no matches found: gym[classic_control]


In [6]:

import gym
from gym.envs.classic_control import rendering
from time import sleep
import minihack


env = gym.make("MiniHack-River-v0")
env.reset() # each reset generates a new environment instance

env.step(1)  # move agent '@' north

viewer = rendering.SimpleImageViewer()
viewer.imshow()
env.render(mode='rgb_array')# each reset generates a new environment instance



ImportError: cannot import name 'rendering' from 'gym.envs.classic_control' (/Users/gltch/Documents/code/github/RL_Project/.cenv/lib/python3.9/site-packages/gym/envs/classic_control/__init__.py)